# Function Calling with Gemini

In [4]:
from google import genai
client = genai.Client()

In [5]:
import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [6]:
get_current_weather("Bangalore")

{'City Name': 'Bangalore',
 'weather': 'mist',
 'temperature': 294.47,
 'unit': 'kelvin'}

### Tool with LLM

- we need to provide the metadata about the tools/tool to the llm
- metadata needs to have 3 components
    - name of the function/tool - should be descriptive, clear
    - description of the function - clear with a sample example
    - arguments: args and its description with example
- OPENAPI format to define metadata

In [7]:
weather_tool = [{"name":"get_current_weather",
                 "description":"this function is used to get/fetch the current weather information for any given city.",
                 "parameters":{"type":"object",
                               "properties":{"city":{"type":'string','description':'name of any location/city. e.g. mumbai, new york'}},
                               "required":['city'],},
                               }]

In [8]:
from google.genai import types
import json
tools = types.Tool(function_declarations=weather_tool)
tool_map = {"get_current_weather":get_current_weather}

config = types.GenerateContentConfig(tools=[tools],
                                     automatic_function_calling=types.FunctionCallingConfig(mode="AUTO"))

In [9]:
def generate_content(prompt:str):
    content = [types.Content(role='user',parts=[types.Part(text=prompt)])]

    first_response = client.models.generate_content(model='gemini-2.0-flash',
                                                    contents=content,
                                                    config=config)
    if first_response.candidates[0].content.parts[0].function_call:
        print("LLM decided to make function call \n",first_response.candidates[0].content.parts[0].function_call)
        content.append(first_response.candidates[0].content)

        tool_call = first_response.candidates[0].content.parts[0].function_call
        tool_name = tool_call.name
        tool_args = tool_call.args
        function_to_ex = tool_map[tool_name]
        tool_output = function_to_ex(**tool_args)

        tool_resp = types.Part.from_function_response(name=tool_name,response={"result":tool_output})

        content.append(types.Content(role='user',parts=[tool_resp]))

        second_response = client.models.generate_content(model='gemini-2.0-flash',contents=content)
        return second_response.text
    else:
        return first_response.text

In [10]:
generate_content("explain quantum computing in 1 line")

'Quantum computing uses quantum mechanics to solve complex problems that are beyond the reach of classical computers.\n'

In [11]:
results = generate_content("what is the weather in Dubai today")
results

LLM decided to make function call 
 id=None args={'city': 'Dubai'} name='get_current_weather'


'The weather in Dubai today is clear sky with a temperature of 298.11 Kelvin.'

# Parallel and Multi Tool Calling

In [12]:
!pip install wikipedia --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [13]:
import wikipedia

def get_wikipedia_summary(query:str)->str:
    response = wikipedia.summary(query)
    return response

In [14]:
print(get_wikipedia_summary("Capital city of Egypt"))

New Administrative Capital (NAC) (Arabic: العاصمة الإدارية الجديدة, romanized: al-ʿĀṣima al-ʾIdārīya al-Gadīda) is a new urban community east of New Cairo in Cairo Governorate, Egypt. It is a satellite city of the nation's original capital city, Cairo. As of May 2023, 14 ministries and government entities have been relocated there. On 2 April 2024, president Abdel Fattah al-Sisi took the constitutional oath for a third consecutive term in office, officially inaugurating the New Administrative Capital as the seat of government.
Plans for the new capital were announced by the then Egyptian housing minister Mostafa Madbouly at the Egypt Economic Development Conference on 13 March 2015.  The capital city is considered one of the projects for economic development, and is part of a larger initiative called Egypt Vision 2030.
Over the years attempts were made to give the city a proper name, other than the New Administrative Capital. A competition was launched on the new capital's website to c

In [15]:
tool_def = [{"name":"get_current_weather",
                 "description":"this function is used to get/fetch the current weather information for any given city.",
                 "parameters":{"type":"object",
                               "properties":{"city":{"type":'string','description':'name of any location/city. e.g. mumbai, new york'}},
                               "required":['city'],},
                               },
            
            {"name":"get_wikipedia_summary",
                 "description":"this function is used to general and historical information about locations, people, places from wikipedia.",
                 "parameters":{"type":"object",
                               "properties":{"query":{"type":'string','description':'topic to search on wikipedia to get summary, e.g. capital of USA'}},
                               "required":['query'],},
                               }]

In [16]:
from google.genai import types
import json
tools = types.Tool(function_declarations=tool_def)
tool_map = {"get_current_weather":get_current_weather,
            "get_wikipedia_summary":get_wikipedia_summary}

config = types.GenerateContentConfig(tools=[tools],
                                     automatic_function_calling=types.FunctionCallingConfig(mode="AUTO"))

In [24]:
def generate_content(prompt:str):
    content = [types.Content(role='user',parts=[types.Part(text=prompt)])]

    while True:
        response = client.models.generate_content(model='gemini-2.0-flash',
                                                    contents=content,
                                                    config=config)
        if response.candidates[0].content.parts[0].function_call:
            print("LLM decided to make function call \n",response.candidates[0].content.parts)
            content.append(response.candidates[0].content)

            tool_calls = response.candidates[0].content.parts

            for tool_call in tool_calls:
                tool_name = tool_call.function_call.name
                tool_args = tool_call.function_call.args
                function_to_ex = tool_map[tool_name]
                tool_output = function_to_ex(**tool_args)
                tool_resp = types.Part.from_function_response(name=tool_name,response={"result":tool_output})
                content.append(types.Content(role='user',parts=[tool_resp]))
        else:
            break
    return response.text

In [25]:
generate_content("explain quantum computing in 1 line")

'Quantum computing uses quantum mechanics to solve complex problems faster than classical computers.\n'

In [26]:
generate_content("what is weather in delhi today")

LLM decided to make function call 
 [Part(
  function_call=FunctionCall(
    args={
      'city': 'delhi'
    },
    name='get_current_weather'
  )
)]


'OK. The current weather in Delhi is haze with a temperature of 296.2 Kelvin.'

In [27]:
# parallel function calls of the same function
generate_content("what is the weather in Singapore and Jakarta today.")

LLM decided to make function call 
 [Part(
  function_call=FunctionCall(
    args={
      'city': 'Singapore'
    },
    name='get_current_weather'
  )
), Part(
  function_call=FunctionCall(
    args={
      'city': 'Jakarta'
    },
    name='get_current_weather'
  )
)]


'The weather in Singapore is broken clouds with a temperature of 303.48 Kelvin. The weather in Jakarta is few clouds with a temperature of 306.79 Kelvin.'

In [29]:
# parallel function calls of different function
results = generate_content("what is the weather in Singapore and tell me more about singapore city.")
print(results)

LLM decided to make function call 
 [Part(
  function_call=FunctionCall(
    args={
      'city': 'Singapore'
    },
    name='get_current_weather'
  )
), Part(
  function_call=FunctionCall(
    args={
      'query': 'Singapore city'
    },
    name='get_wikipedia_summary'
  )
)]
The current weather in Singapore is broken clouds with a temperature of 303.48 Kelvin.

Singapore is an island country and city-state in Southeast Asia. It's located off the southern tip of the Malay Peninsula. In its early history, it was a maritime emporium called Temasek. It was later established as a British trading post in 1819 by Stamford Raffles. Singapore gained independence in 1965 and has since developed into a highly developed country with the highest PPP-adjusted GDP per capita in the world. It's a major hub for aviation, finance, and maritime shipping. Singapore is known for its high quality of life, education, healthcare, and infrastructure. It has a multicultural population and four official lan

In [33]:
# sequential function calls of different function
results = generate_content("what is the weather in the capital city of Tripura")
print(results)

LLM decided to make function call 
 [Part(
  function_call=FunctionCall(
    args={
      'query': 'capital of Tripura'
    },
    name='get_wikipedia_summary'
  )
)]
LLM decided to make function call 
 [Part(
  function_call=FunctionCall(
    args={
      'city': 'Agartala'
    },
    name='get_current_weather'
  )
)]
The current weather in Agartala, the capital of Tripura, is mist with a temperature of 301.16 Kelvin.

